# Optimization and Black-Litterman Model

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
# from HW-2A: Problem 1-A-3, locate sector returns
factor = pd.read_excel('HW2.xlsx', sheet_name = 'SPX_DXY_MonthlyReturns')
factor.columns = ['Date', 'S&P 500 Index', 'US Dollar Index']
factor.index = factor.Date
factor.drop('Date', axis = 1, inplace = True)
df = pd.read_excel('HW2.xlsx', sheet_name = 'SP500Stocks',header = 1)
df0 = df.copy()
df0.index = df.NAME
df0.drop(['NAME','TICKER','SECTOR','Market Cap ($ Mil)'], axis = 1, inplace = True)
df1 = df0.T
num_stocks = df1.shape[1]
num_months = df1.shape[0]
beta_mat = np.empty((num_stocks, 1 + len(factor.columns),))
beta_mat[:] = np.nan
beta_mat[:,0] = 1
for i in range(0,num_stocks):
    y = df1.iloc[:,i]
    x = sm.add_constant(factor.to_numpy())
    model = sm.OLS(y,x,missing='drop').fit()
    beta_mat[i,1] = model.params[1]
    beta_mat[i,2] = model.params[2]    
df2 = df.iloc[:,:3]
df2.drop('TICKER',axis = 1,inplace = True)
sector_set = [np.nan]*len(df2.SECTOR.unique())
for i in range(len(df2.SECTOR.unique())):
    sector_set[i] = df2.SECTOR.unique()[i]
sector_set.remove('UTIL')
num_sector = len(sector_set)
l = np.zeros((num_stocks,num_sector))
df_sector = pd.DataFrame(l)
df_sector.columns = sector_set
df_sector.index = df2.NAME
for i in range(num_stocks):    
    for j in range(len(df_sector.columns)):        
        if df2['SECTOR'].iloc[i] != df_sector.columns[j]:
            continue
        if df2['SECTOR'].iloc[i] == df_sector.columns[j]:
            df_sector.iloc[i][j] = 1        
df_sector.head() 
sector_exposure = df_sector.to_numpy()
df3 = df.iloc[:,:4]
df3.drop('TICKER',axis = 1,inplace = True)
df3['log(Market Cap)'] = np.log(df3['Market Cap ($ Mil)'])
mean = df3.groupby('SECTOR')['log(Market Cap)'].mean()
std = df3.groupby('SECTOR')['log(Market Cap)'].std()
size_exposure = np.empty((num_stocks,1))
for i in range(num_stocks):
    sector = df3['SECTOR'].iloc[i]
    size_exposure[i] = (df3['log(Market Cap)'].iloc[i] - mean[sector])/ std[sector]
factor_list = [factor.columns[0],factor.columns[1]]
for i in range(len(sector_set)):
    factor_list.append(sector_set[i])
factor_list.append('Market Cap')
beta_mat = np.concatenate((beta_mat,sector_exposure,size_exposure), axis = 1)
df_beta_mat = pd.DataFrame(beta_mat)
l = ['Constant']
f = np.concatenate((l,factor_list), axis = 0)
df_beta_mat.columns = f
df_beta_mat.index = df3.NAME
num_factor = len(factor_list)
factor_ret = np.empty((num_factor,num_months))
factor_ret[:] = None
for t in range(num_months):
    y = df0.iloc[:,t]
    x = beta_mat
    model = sm.OLS(y,x,missing = 'drop').fit()
    factor_ret[:,t] = model.params[1:]
factor_ret = pd.DataFrame(factor_ret)    
factor_ret.columns = df0.columns
factor_ret.index = factor_list
factor_ret

,2001-07-31 00:00:00,2001-08-31 00:00:00,2001-09-30 00:00:00,2001-10-31 00:00:00,2001-11-30 00:00:00,2001-12-31 00:00:00,2002-01-31 00:00:00,2002-02-28 00:00:00,2002-03-31 00:00:00,2002-04-30 00:00:00,...,2018-03-31 00:00:00,2018-04-30 00:00:00,2018-05-31 00:00:00,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00,2018-10-31 00:00:00,2018-11-30 00:00:00,2018-12-31 00:00:00
S&P 500 Index,-6.862857,-6.531107,-20.779600,9.164590,11.163984,1.215029,-4.323136,-5.533380,6.677938,-8.524798,...,-1.541033,-2.245750,2.079768,-2.869671,1.789404,-1.263999,-0.475853,-7.030800,0.490195,-3.176790
US Dollar Index,1.148899,-3.814941,-4.427043,5.534274,5.878303,-0.502738,-1.612277,-3.514096,-0.359773,-4.333348,...,-0.842876,0.077256,3.955822,1.263941,0.014721,1.749081,-1.058467,3.362589,1.699337,0.487357
INDU,9.781195,0.346426,9.998076,-3.630006,2.105712,0.565578,7.164433,10.661489,-10.552080,3.759445,...,-4.432178,-4.305730,1.022698,-2.894414,4.583613,0.885673,2.033739,-8.910813,1.470327,-5.440288
HLTH,4.397612,0.190909,8.834765,2.879946,0.536815,-3.196261,0.527301,2.695108,-8.177159,-0.901076,...,-5.581573,-1.403980,0.652143,0.176648,3.401090,2.786344,3.402252,-8.664647,1.826115,-4.603050
INFT,7.325754,-0.635126,8.745651,8.241545,3.268029,-1.688333,7.519178,0.528758,-6.466756,-2.971178,...,-4.275920,-0.903985,2.423637,-0.614386,-1.094338,5.066053,1.080130,-6.393504,-2.400866,-2.210402
TCOM,4.825124,-0.833424,5.572955,3.509324,0.349600,-2.472622,1.867889,9.126370,-9.292503,1.268775,...,-8.487334,-0.950837,1.476050,9.945045,-5.979237,0.840789,3.112886,-0.741794,-4.752180,-4.661255
DSCR,12.194093,-0.846280,6.145132,0.473640,4.318661,3.370005,10.697396,8.667919,-11.619652,9.601583,...,-5.087975,0.796624,0.358355,1.071403,-1.414894,0.579908,1.160565,-4.720328,-1.780387,-3.903042
FINA,7.337022,-0.366846,17.953474,-9.750611,-0.611098,-1.179031,4.286246,7.722149,-9.511225,4.989156,...,-5.830681,-0.698454,-2.097977,-2.526363,0.872308,-0.221929,-1.441822,-2.727631,-0.899673,-5.496160
MATS,9.080813,2.162124,10.472454,-1.789567,4.865412,-2.591446,6.720628,9.879490,-7.862185,0.769461,...,-6.761645,-0.783629,2.462243,-0.256819,-0.305439,-2.074810,0.625090,-5.299211,1.504166,-2.081822
REAL,3.769479,1.999704,12.708085,-6.354562,0.613596,-2.717193,1.122977,3.366614,-6.494473,3.682304,...,-0.057924,-1.057510,2.456788,3.748026,-1.893627,1.655226,-2.214345,-0.162410,2.529691,-2.285940


In [3]:
sector_ret = factor_ret.iloc[2:12,:]
sector_ret

,2001-07-31 00:00:00,2001-08-31 00:00:00,2001-09-30 00:00:00,2001-10-31 00:00:00,2001-11-30 00:00:00,2001-12-31 00:00:00,2002-01-31 00:00:00,2002-02-28 00:00:00,2002-03-31 00:00:00,2002-04-30 00:00:00,...,2018-03-31 00:00:00,2018-04-30 00:00:00,2018-05-31 00:00:00,2018-06-30 00:00:00,2018-07-31 00:00:00,2018-08-31 00:00:00,2018-09-30 00:00:00,2018-10-31 00:00:00,2018-11-30 00:00:00,2018-12-31 00:00:00
INDU,9.781195,0.346426,9.998076,-3.630006,2.105712,0.565578,7.164433,10.661489,-10.552080,3.759445,...,-4.432178,-4.305730,1.022698,-2.894414,4.583613,0.885673,2.033739,-8.910813,1.470327,-5.440288
HLTH,4.397612,0.190909,8.834765,2.879946,0.536815,-3.196261,0.527301,2.695108,-8.177159,-0.901076,...,-5.581573,-1.403980,0.652143,0.176648,3.401090,2.786344,3.402252,-8.664647,1.826115,-4.603050
INFT,7.325754,-0.635126,8.745651,8.241545,3.268029,-1.688333,7.519178,0.528758,-6.466756,-2.971178,...,-4.275920,-0.903985,2.423637,-0.614386,-1.094338,5.066053,1.080130,-6.393504,-2.400866,-2.210402
TCOM,4.825124,-0.833424,5.572955,3.509324,0.349600,-2.472622,1.867889,9.126370,-9.292503,1.268775,...,-8.487334,-0.950837,1.476050,9.945045,-5.979237,0.840789,3.112886,-0.741794,-4.752180,-4.661255
DSCR,12.194093,-0.846280,6.145132,0.473640,4.318661,3.370005,10.697396,8.667919,-11.619652,9.601583,...,-5.087975,0.796624,0.358355,1.071403,-1.414894,0.579908,1.160565,-4.720328,-1.780387,-3.903042
FINA,7.337022,-0.366846,17.953474,-9.750611,-0.611098,-1.179031,4.286246,7.722149,-9.511225,4.989156,...,-5.830681,-0.698454,-2.097977,-2.526363,0.872308,-0.221929,-1.441822,-2.727631,-0.899673,-5.496160
MATS,9.080813,2.162124,10.472454,-1.789567,4.865412,-2.591446,6.720628,9.879490,-7.862185,0.769461,...,-6.761645,-0.783629,2.462243,-0.256819,-0.305439,-2.074810,0.625090,-5.299211,1.504166,-2.081822
REAL,3.769479,1.999704,12.708085,-6.354562,0.613596,-2.717193,1.122977,3.366614,-6.494473,3.682304,...,-0.057924,-1.057510,2.456788,3.748026,-1.893627,1.655226,-2.214345,-0.162410,2.529691,-2.285940
STPL,6.474854,0.586196,6.042393,-0.455695,2.977548,-3.028078,4.530565,6.709852,-9.232969,0.912770,...,-5.084995,-5.455916,-2.193325,2.916290,0.567354,-0.416350,0.067406,-0.976214,-2.040011,-6.061369
ENER,5.315452,-3.381388,4.087406,7.848457,-8.522760,3.799435,-1.851951,9.980194,-2.622675,1.177210,...,-0.680842,9.672145,3.682521,0.755149,-0.401137,-4.342493,2.554841,-10.515252,-7.983625,-8.856122


In [4]:
num_sec = len(sector_ret)
num_sec

10

In [5]:
num_month = len(sector_ret.columns)
num_month

210

## A

In [6]:
# equilibrium expected monthly return (%)
Pi = sector_ret.T.mean()
Pi

INDU    0.208777
HLTH    0.410648
INFT    0.327142
TCOM    0.097279
DSCR    0.310634
FINA   -0.160219
MATS    0.260696
REAL    0.363441
STPL    0.131597
ENER    0.174261
dtype: float64

In [7]:
# monthly return covariance
Sigma = sector_ret.T.cov()
Sigma

,INDU,HLTH,INFT,TCOM,DSCR,FINA,MATS,REAL,STPL,ENER
INDU,19.021144,13.423490,13.725328,13.482213,17.599473,17.212288,16.623234,9.201910,12.184452,13.187322
HLTH,13.423490,17.159269,12.883104,12.571509,13.985306,14.551422,12.397148,9.244299,11.048957,8.965231
INFT,13.725328,12.883104,19.768509,14.629161,14.075626,12.706897,14.431778,6.267675,10.351698,13.093622
TCOM,13.482213,12.571509,14.629161,20.265704,15.550637,14.198118,14.447004,7.090559,12.207699,13.248884
DSCR,17.599473,13.985306,14.075626,15.550637,24.475737,18.236456,16.343521,12.627686,13.870493,10.203874
FINA,17.212288,14.551422,12.706897,14.198118,18.236456,25.334354,15.920561,11.461596,13.259095,10.835000
MATS,16.623234,12.397148,14.431778,14.447004,16.343521,15.920561,20.040624,8.084687,11.182951,15.269886
REAL,9.201910,9.244299,6.267675,7.090559,12.627686,11.461596,8.084687,20.558447,7.977990,2.251384
STPL,12.184452,11.048957,10.351698,12.207699,13.870493,13.259095,11.182951,7.977990,12.958384,7.640241
ENER,13.187322,8.965231,13.093622,13.248884,10.203874,10.835000,15.269886,2.251384,7.640241,36.208822


In [8]:
import scipy
mon_vol = 10/12
def maxr_func(w):
    maxr_func = -np.dot(Pi.T,w)
    return maxr_func
w = np.ones([num_sec])/num_sec         
c_ = ({'type':'eq', 'fun': lambda w: sum(w)-1}, 
       {'type':'ineq', 'fun': lambda w: -np.dot(np.dot(w,Sigma),w) + mon_vol**2 }) 
optimized = scipy.optimize.minimize(maxr_func, w, method='SLSQP', constraints=c_) 
w1 = optimized.x
w1.round(5)

array([-0.03043,  0.07654,  0.14751, -0.04122, -0.26504, -0.11363,
        0.07064,  0.33795,  0.68267,  0.13501])

In [9]:
weight = pd.DataFrame(w1*100, columns = ['optim_w(%)'], index = sector_ret.index)
weight

,optim_w(%)
INDU,-3.043144
HLTH,7.653560
INFT,14.751369
TCOM,-4.121627
DSCR,-26.503952
FINA,-11.363451
MATS,7.064189
REAL,33.794998
STPL,68.266709
ENER,13.501349


## B

## 1)

In [10]:
tau = 0.05

In [11]:
# view1: Sector ENER will have return of 0.20% in next month.
tmp_pos = 9
port_v1 = [0]*num_sec
port_v1[tmp_pos] = 1
port_v1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [12]:
q1 = 0.2
print('View 1: equlibrium return = %.2f%%' % np.dot(port_v1,Pi), ', active view return = %.2f%%.\n' % q1)

View 1: equlibrium return = 0.17% , active view return = 0.20%.



In [13]:
# view2: Sector FINA will outperform Sector INDU by 0.05% in next month.
tmp_long = 5
tmp_short = 0
port_v2 = [0]*num_sec
port_v2[tmp_long] = 1
port_v2[tmp_short] = -1
port_v2

[-1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [14]:
q2 = 0.05
print('View 2: equlibrium return = %.2f%%' % np.dot(port_v2,Pi), ', active view return = %.2f%%.\n' % q2)

View 2: equlibrium return = -0.37% , active view return = 0.05%.



In [15]:
# view3: Sector INFT and Sector HLTH will outperform Sector REAL and Sector STPL by 0.30% in next month.  (Assuming equal weighting on both long and short legs)
tmp_long1 = 2
tmp_long2 = 1
tmp_short1 = 7
tmp_short2 = 8
port_v3= [0]*num_sec
port_v3[tmp_long1] = 0.5
port_v3[tmp_long2] = 0.5
port_v3[tmp_short1] = -0.5
port_v3[tmp_short2] = -0.5
port_v3

[0, 0.5, 0.5, 0, 0, 0, 0, -0.5, -0.5, 0]

In [16]:
q3 = 0.3
print('View 3: equlibrium return = %.2f%%' % np.dot(port_v3,Pi), ', active view return = %.2f%%.\n' % q3)

View 3: equlibrium return = 0.12% , active view return = 0.30%.



In [17]:
P = np.matrix([port_v1,port_v2,port_v3])
P

matrix([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ],
        [-1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0.5,  0.5,  0. ,  0. ,  0. ,  0. , -0.5, -0.5,  0. ]])

In [18]:
# show the three view portfolios in DataFrame
df = pd.DataFrame(P.T, index = sector_ret.index, columns = ['View1','View2','View3'])
df

,View1,View2,View3
INDU,0.0,-1.0,0.0
HLTH,0.0,0.0,0.5
INFT,0.0,0.0,0.5
TCOM,0.0,0.0,0.0
DSCR,0.0,0.0,0.0
FINA,0.0,1.0,0.0
MATS,0.0,0.0,0.0
REAL,0.0,0.0,-0.5
STPL,0.0,0.0,-0.5
ENER,1.0,0.0,0.0


In [19]:
# View Return Vector Q
Q = [q1,q2,q3]
Q

[0.2, 0.05, 0.3]

In [20]:
# View Uncertainty Matrix Omega
port_v1 = np.matrix(port_v1)
port_v2 = np.matrix(port_v2)
port_v3 = np.matrix(port_v3)
omega1 = tau*np.dot(np.dot(port_v1, Sigma), port_v1.T)
omega2 = tau*np.dot(np.dot(port_v2, Sigma), port_v2.T)
omega3 = tau*np.dot(np.dot(port_v3, Sigma), port_v3.T)
Omega = np.zeros((3,3),float)
np.fill_diagonal(Omega,[omega1,omega2,omega3])
Omega

array([[1.81044112, 0.        , 0.        ],
       [0.        , 0.49654607, 0.        ],
       [0.        , 0.        , 0.47926927]])

In [21]:
# Black-Litterman Results

In [22]:
C = tau*Sigma
H = np.dot(np.dot(P.T, np.linalg.inv(Omega)), P) + np.linalg.inv(C)
nu = np.dot((np.dot(np.dot(P.T, np.linalg.inv(Omega)),Q) + np.dot(np.linalg.inv(C),Pi)), np.linalg.inv(H))
nu

matrix([[0.20280387, 0.49093136, 0.39568801, 0.15880032, 0.33473656,
         0.03417581, 0.28936459, 0.34103794, 0.15887594, 0.19508078]])

In [23]:
# compare the equlibrium monthly return Pi and the updated expected return
df1 = pd.DataFrame(Pi, index = sector_ret.index, columns = ['Exp_Ret(%)'])
df1['Updated_Exp_Ret(%)'] = nu.T
df1

,Exp_Ret(%),Updated_Exp_Ret(%)
INDU,0.208777,0.202804
HLTH,0.410648,0.490931
INFT,0.327142,0.395688
TCOM,0.097279,0.158800
DSCR,0.310634,0.334737
FINA,-0.160219,0.034176
MATS,0.260696,0.289365
REAL,0.363441,0.341038
STPL,0.131597,0.158876
ENER,0.174261,0.195081


## 2)

In [24]:
# The updated FINA performs much better than original expectation.
# In view1, ENER's expected return and updated expected return are lower than analyst's subjective views.
# Updated views improve updated returns which are higher than original expected return

## 3)

In [25]:
mon_vol = 10/12
def maxr_func(w):
    maxr_func = -np.dot(df1['Updated_Exp_Ret(%)'].T,w)
    return maxr_func
w = np.ones([num_sec])/num_sec         
c_ = ({'type':'eq', 'fun': lambda w: sum(w)-1}, 
       {'type':'ineq', 'fun': lambda w: -np.dot(np.dot(w,np.linalg.inv(H) + Sigma),w) + mon_vol**2 }) 
optimized = scipy.optimize.minimize(maxr_func, w, method='SLSQP', constraints=c_) 
w2 = optimized.x
w2.round(5)

array([-0.03085,  0.07493,  0.14557, -0.04088, -0.26327, -0.11223,
        0.07014,  0.33672,  0.67908,  0.1408 ])

In [26]:
# compare the updated optimal portfolio (the weight vector) with the solution from A 
weight['updated_optim_w(%)'] = w2*100
weight

,optim_w(%),updated_optim_w(%)
INDU,-3.043144,-3.085026
HLTH,7.653560,7.493047
INFT,14.751369,14.556660
TCOM,-4.121627,-4.088118
DSCR,-26.503952,-26.327035
FINA,-11.363451,-11.222836
MATS,7.064189,7.013923
REAL,33.794998,33.671606
STPL,68.266709,67.907901
ENER,13.501349,14.079878
